In [34]:
import anthropic
import base64
import pandas as pd
import os
import base64
import re
import numpy as np

In [35]:
macronutrients_instruction = '''Examine the provided meal image to analyze and estimate its nutritional content accurately. Focus on determining the amounts of simple sugars (like industrial sugar and honey), 
complex sugars (such as starch and whole grains), proteins, fats, and dietary fibers (found in fruits and vegetables), all in grams. Also estimate the total weight of the meal in grams.
To assist in accurately gauging the scale of the meal, a 1 Swiss Franc coin, which has a diameter of 23.22 mm, may be present in the picture. 
Use the size of this coin as a reference to estimate the size of the meal and the amounts of the nutrients more precisely. 
Provide your assessment of each nutritional component in grams. All estimates should be given as a single whole number. If there is no coin in the picture or the meal is covered partially, estimate anyways.
Format your response as follows:
- Simple sugars (g): 
- Complex sugars (g): 
- Proteins (g): 
- Fats (g): 
- Dietary fibers (g): 
- Weight (g): 
- Explanation: 

Example response:
Simple sugars (g): 40
Complex sugars (g): 60
Proteins (g): 25
Fats (g): 30
Dietary fibers (g): 5 
Weight (g): 750
Explanation: The pizza and cola meal, with its refined crust and toppings, is rich in carbs, fats, and proteins. The cola boosts the meal's simple sugars. 
The 1 Swiss Franc coin helps estimate the pizza at 30 cm diameter and the cola at 330 ml, indicating a significant blood sugar impact.'''

In [36]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def parse_nutritional_info(text):
    pattern = r'(Simple sugars \(g\)|Complex sugars \(g\)|Proteins \(g\)|Fats \(g\)|Dietary fibers \(g\)|Weight \(g\)):\s*(\d+)'
    matches = re.findall(pattern, text)
    nutritional_info = {match[0]: int(match[1]) for match in matches}
    simple_sugars = nutritional_info.get('Simple sugars (g)', 0)
    complex_sugars = nutritional_info.get('Complex sugars (g)', 0)
    proteins = nutritional_info.get('Proteins (g)', 0)
    fats = nutritional_info.get('Fats (g)', 0)
    dietary_fibers = nutritional_info.get('Dietary fibers (g)', 0)
    weight = nutritional_info.get('Weight (g)', 0)
    return simple_sugars, complex_sugars, proteins, fats, dietary_fibers, weight


image_path = "/Users/jwolber/Documents/Python_Projects/Glucovision/diabetes_subset_pictures-glucose-food-insulin/001/food_pictures/001.jpg"
base64_image = encode_image(image_path)




In [37]:
client = anthropic.Anthropic(api_key="your-key-here")


In [48]:
import os
from PIL import Image

def compress_image(input_path, output_path, quality=85):
    image = Image.open(input_path)
    image.save(output_path, "JPEG", quality=quality)

def compress_images_in_directory(directory, output_directory, quality):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for filename in os.listdir(directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(directory, filename)
            output_path = os.path.join(output_directory, filename)
            compress_image(input_path, output_path, quality)
            print(f"Compressed {input_path} and saved to {output_path}")

input_directory = "/Users/jwolber/Documents/Python_Projects/Glucovision/diabetes_subset_pictures-glucose-food-insulin/008/food_pictures"
output_directory = "/Users/jwolber/Documents/Python_Projects/Glucovision/diabetes_subset_pictures-glucose-food-insulin/008/food_pictures"
compress_images_in_directory(input_directory, output_directory, quality=20)


Compressed /Users/jwolber/Documents/Python_Projects/Glucovision/diabetes_subset_pictures-glucose-food-insulin/008/food_pictures/002.jpg and saved to /Users/jwolber/Documents/Python_Projects/Glucovision/diabetes_subset_pictures-glucose-food-insulin/008/food_pictures/002.jpg
Compressed /Users/jwolber/Documents/Python_Projects/Glucovision/diabetes_subset_pictures-glucose-food-insulin/008/food_pictures/003.jpg and saved to /Users/jwolber/Documents/Python_Projects/Glucovision/diabetes_subset_pictures-glucose-food-insulin/008/food_pictures/003.jpg
Compressed /Users/jwolber/Documents/Python_Projects/Glucovision/diabetes_subset_pictures-glucose-food-insulin/008/food_pictures/017.jpg and saved to /Users/jwolber/Documents/Python_Projects/Glucovision/diabetes_subset_pictures-glucose-food-insulin/008/food_pictures/017.jpg
Compressed /Users/jwolber/Documents/Python_Projects/Glucovision/diabetes_subset_pictures-glucose-food-insulin/008/food_pictures/001.jpg and saved to /Users/jwolber/Documents/Pyth

In [49]:
for patient in ['001', '002', '004', '006', '007', '008']:
    print(f"Processing patient {patient}")
    food_data_path = f"/Users/jwolber/Documents/Python_Projects/Glucovision/diabetes_subset_pictures-glucose-food-insulin/{patient}/food.csv"
    food_data = pd.read_csv(food_data_path)
    food_data[['simple_sugars', 'complex_sugars', 'proteins', 'fats', 'dietary_fibers', 'weight', 'message']] = 0
    for i, row in food_data.iterrows():
        image_path = f"/Users/jwolber/Documents/Python_Projects/Glucovision/diabetes_subset_pictures-glucose-food-insulin/{patient}/food_pictures/{row['picture']}"
        base64_image = encode_image(image_path)
        response = client.messages.create(
            model="claude-3-5-sonnet-20240620",
            max_tokens=1024,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": "image/jpeg",  
                                "data": base64_image,
                            },
                        },
                        {
                            "type": "text",
                            "text": macronutrients_instruction
                        }
                    ],
                }
            ],
        )
        message = response.content[0].text
        try:
            parsed_info = parse_nutritional_info(message)
            print(parsed_info)
            food_data.loc[i, ['simple_sugars', 'complex_sugars', 'proteins', 'fats', 'dietary_fibers', 'weight']] = parsed_info
            food_data.loc[i, 'message'] = message
        except:
            print(f"Picture {row['picture']} for patient {patient} could not be annotated")
    food_data.to_csv(f'/Users/jwolber/Documents/Python_Projects/Glucovision/sonnet/{patient}.csv', index=False)

Processing patient 008
(2, 50, 15, 8, 3, 300)
(2, 45, 10, 3, 4, 280)
(3, 65, 25, 15, 2, 350)
(0, 2, 1, 1, 0, 250)
(8, 30, 25, 15, 12, 450)
(15, 85, 20, 25, 8, 450)
(5, 70, 30, 20, 8, 400)
(5, 30, 8, 10, 2, 200)
(2, 45, 30, 20, 3, 300)
(2, 30, 3, 1, 4, 150)
(25, 30, 8, 15, 2, 300)
(1, 30, 4, 1, 2, 80)
(2, 55, 30, 15, 3, 300)
(5, 40, 20, 15, 3, 300)
(1, 20, 5, 2, 1, 30)
